In [1]:
%run "library.py"

[NbConvertApp] Converting notebook library.ipynb to python
[NbConvertApp] Writing 2792 bytes to library.py


In [2]:
from sklearn import preprocessing
from sklearn.lda import LDA
from sklearn import svm, cross_validation
from sklearn.cross_validation import KFold
from sklearn import linear_model, svm
import numpy as np


In [3]:
train_indices, test_indices = [], []
for train_index, test_index in KFold(len(train_cnn), n_folds=5):
    train_indices = train_index
    test_indices = test_index
y_train, y_test = train_labels[train_indices], train_labels[test_indices]
print train_indices.shape, test_indices.shape, len(train_cnn)

(2400,) (600,) 3000


## CNN

In [5]:
train_cnn_normalized = preprocessing.scale(train_cnn, axis=1)
predictions_cnn, svmmodel_cnn, score_cnn = 0, 0, 0

x_train_cnn, x_test_cnn = train_cnn_normalized[train_indices], train_cnn_normalized[test_indices]

svmmodel_cnn = svm.SVC(kernel='linear', probability=True).fit(x_train_cnn,y_train) 
score_cnn = svmmodel_cnn.score(x_test_cnn,y_test) 
predictions_cnn = svmmodel_cnn.predict_proba(x_test_cnn) 
print score_cnn

0.373333333333


In [6]:
#weighted predictions
weight_cnn = 0.479657388
weighted_predictions_cnn = weight_cnn*np.array(predictions_cnn)

## BOW

In [7]:
train_bow_normalized = preprocessing.scale(train_bow, axis=0)
predictions_bow, svmmodel_bow, score_bow = 0, 0, 0

x_train_bow, x_test_bow = train_bow_normalized[train_indices], train_bow_normalized[test_indices]

svmmodel_bow = svm.SVC(kernel='linear', probability=True).fit(x_train_bow,y_train) 
score_bow = svmmodel_bow.score(x_test_bow,y_test) 
predictions_bow = svmmodel_bow.predict_proba(x_test_bow) 
print score_bow

0.115


In [8]:
#weighted predictions
weight_bow = 0.147751606
weighted_predictions_bow = weight_bow*np.array(predictions_bow)

## ATTRIBS

In [9]:
train_attribs_normalized = preprocessing.scale(train_attribs, axis=1)
predictions_attribs, svmmodel_attribs, score_attribs = 0, 0, 0

x_train_attribs, x_test_attribs = train_attribs_normalized[train_indices], train_attribs_normalized[test_indices]

svmmodel_attribs = svm.SVC(kernel='poly',degree = 2, probability=True).fit(x_train_attribs,y_train) 
score_attribs = svmmodel_attribs.score(x_test_attribs,y_test) 
predictions_attribs = svmmodel_attribs.predict_proba(x_test_attribs) 
print score_attribs

0.29


In [10]:
#weighted predictions
weight_attribs =0.372591006
weighted_predictions_attribs = weight_attribs*np.array(predictions_attribs)

## OVERALL

In [11]:
predictions_mean = ( weighted_predictions_cnn + weighted_predictions_bow + weighted_predictions_attribs ) / 3.0
max_indices = np.argmax(predictions_mean,axis=1)
predicted_classes = svmmodel_cnn.classes_[max_indices]

In [12]:
# overall score
sum(predicted_classes == y_test)*1.0/len(y_test)

0.43333333333333335

In [29]:
def get_score(wt):
    wt_cnn = wt[0]
    wt_bow = wt[1]
    wt_attrib = wt[2]
    predictions_mean = ( wpred(predictions_cnn,wt_cnn) + wpred(predictions_bow,wt_bow) + wpred(predictions_attribs,wt_attrib) ) / 3.0
    max_indices = np.argmax(predictions_mean,axis=1)
    predicted_classes = svmmodel_cnn.classes_[max_indices]
    score = sum(predicted_classes == y_test)*1.0/len(y_test)
    print score
    return (1 - score)

In [47]:
res.x

array([ 1.13996196,  0.21358706,  0.47390117])

In [46]:

from scipy.optimize import minimize

#x0 = [0.496763484,0.167026471,0.336210907]
x0 = [0.39,0.42,0.19]
# x0 = [0.43,0.47,0.1]
res = minimize(get_score, x0, method='Powell')

0.418333333333
0.418333333333
0.393333333333
0.025
0.418333333333
0.136666666667
0.418333333333
0.418333333333
0.411666666667
0.415
0.411666666667
0.415
0.418333333333
0.416666666667
0.418333333333
0.418333333333
0.418333333333
0.418333333333
0.338333333333
0.358333333333
0.418333333333
0.42
0.4
0.423333333333
0.421666666667
0.423333333333
0.426666666667
0.426666666667
0.426666666667
0.425
0.425
0.425
0.426666666667
0.406666666667
0.0783333333333
0.426666666667
0.23
0.423333333333
0.428333333333
0.426666666667
0.433333333333
0.431666666667
0.433333333333
0.433333333333
0.43
0.433333333333
0.431666666667
0.433333333333
0.42
0.433333333333
0.416666666667
0.08
0.433333333333
0.296666666667
0.433333333333
0.423333333333
0.428333333333
0.433333333333
0.433333333333
0.435
0.431666666667
0.433333333333
0.433333333333
0.433333333333
0.435
0.405
0.393333333333
0.435
0.421666666667
0.431666666667
0.431666666667
0.426666666667
0.43
0.431666666667
0.435
0.435
0.435
0.435
0.435
0.435
0.435
0.435
0.

## ALL DATA AT ONCE

In [14]:
train_all = np.hstack([train_cnn, train_bow, train_attribs])
train_all_normalized = preprocessing.scale(train_all, axis=0)
predictions_all, svmmodel_all, score_all = 0, 0, 0

x_train_all, x_test_all = train_all_normalized[train_indices], train_all_normalized[test_indices]

svmmodel_all = svm.SVC(kernel='linear', probability=True).fit(x_train_all,y_train) 
score_all = svmmodel_all.score(x_test_all,y_test) 
predictions_all = svmmodel_all.predict_proba(x_test_all) 
print score_all

0.355


## KAGGLE SUBMISSION

In [15]:
def svm_predictions_for(train_vals, test_vals, train_lbls, axs=1):
    train_vals_normalized = preprocessing.scale(train_vals, axis=axs)
    predictions_vals, svmmodel = 0, 0

    svmmodel = svm.SVC(kernel='linear', probability=True).fit(train_vals,train_lbls) 
    predictions_vals = svmmodel.predict_proba(test_vals) 
    return predictions_vals, svmmodel

In [16]:
predictions_cnn_final, svm_cnn_final = svm_predictions_for(train_cnn, test_cnn, train_labels, axs=1)

In [17]:
predictions_bow_final, svm_bow_final = svm_predictions_for(train_bow, test_bow, train_labels, axs=0)

In [18]:
predictions_attribs_final, svm_attribs_final = svm_predictions_for(train_attribs, test_attribs, train_labels, axs=1)

In [19]:
def wpred(predictions, wt):
    return wt*np.array(predictions)

predictions_mean_final = ( wpred(predictions_cnn_final,0.479657388) + wpred(predictions_bow_final,0.147751606) + wpred(predictions_attribs_final,0.372591006) ) / 3.0
max_indices_final = np.argmax(predictions_mean_final,axis=1)
predicted_classes_final = svm_cnn_final.classes_[max_indices_final]
kaggle_output(predicted_classes_final)

[['0000.jpg' 'soccer_field']
 ['0001.jpg' 'rubble']
 ['0002.jpg' 'junk_pile']
 ..., 
 ['0997.jpg' 'reception']
 ['0998.jpg' 'cheese_factory']
 ['0999.jpg' 'pasture']]
